## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import json
from nocasedict import NocaseDict
import re
import requests
import yaml
import math
import simplejson

## 2. Establish Key Variables

In [2]:
# LGBF_DataFile_Url = input("Input Web Address of LGBF Data File")
# secrets_location = input("Enter the full file path for your secrets file"

LGBF_DataFile_Url = "https://www.improvementservice.org.uk/__data/assets/excel_doc/0021/23844/2019-20-lgbf-raw-data-may_2021_refresh.xlsx"
secrets_location = "P:\\My Documents\\Github\\Supporting Files\\secrets.yml"
secrets = open(secrets_location)
secrets = yaml.load(secrets,Loader=yaml.FullLoader)['SharePoint_LGBF']
SP_clientsecret = secrets['client_secret']
SP_tenantid = secrets['tenant_id']
SP_tenant = "stirlingcounciluk"
SP_clientid = secrets['client_id'] + '@' + SP_tenantid

## 3. Establish Functions

In [3]:
# Establish function to unpivot data as this will be repeated for each sheet. This function only works with the LGBF data file because the layout of the 4 sheets is the same.
def unpivotData (df) :
    # Promote Headers
    new_headers = df.iloc[0]
    df = df[1:]
    df.columns = new_headers
    # Unpivot Data - !!!! This is creating additional blank rows for num/den dataframes - look into fix. Currently not causing any real issues (as far as I can tell from basic checks of the resulting csv) as additional blank rows can just be dropped!!!!
    df = df.melt(id_vars = ['GSS Code', 'Local Authority'],var_name = 'Attribute',value_name = 'Value')
    # For some reason only the indicators coded as corp contain a space between the code text and the number. Added a step here to correct it as it applies to all dataframes and causes an issue with splitting code and period from the single string format it is supplied in due to inconsistency.
    df['Attribute'] = df['Attribute'].str.replace('CORP ','CORP')
    return df

# Establish function that provides a signed difference between two values.
def distance(Current, Previous): 
    return (max(Previous, Current) - min(Previous, Current)) * (-1 if Previous > Current else 1)


## 4. Establish Key Dataframes
Get each sheet from the excel file and load them into their own distinct dataframes. Basic transformation steps will be applied here to ensure each dataframe has appropriate headers, data is unpivoted etc. More in depth transformation will be handled later.

### 4.1 Establish Dataframes from LGBF Raw Data XLSX File

In [4]:
# Collect the excel file from the url/location provided
dataFile = pd.ExcelFile(LGBF_DataFile_Url)
# Write each sheet within the file to it's own data frame. These will eventually be combined into a single more usable dataframe. When rerunning this notebook be careful of the sheet names the Improvement Service have chosen (in particular trailing spaces).
cashIndicators = pd.read_excel(dataFile,'Cash Indicators')
cashIndicators_NumeratorDenominator = pd.read_excel(dataFile,' Cash Num-Den_Indi')
realIndicators = pd.read_excel(dataFile,'Real Indicators ')
realIndicators_NumeratorDenominator = pd.read_excel(dataFile,'Real Num_Den_Indi')

# Output realIndicators to give an example of the output from this step.
realIndicators

,Unnamed: 0,All costs are real values,CHN1 - Cost per primary school pupil,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,CLIM2 - CO2 emissions area wide: emissions within scope of LA per capita,Unnamed: 782,Unnamed: 783,Unnamed: 784,Unnamed: 785,Unnamed: 786,Unnamed: 787,Unnamed: 788,Unnamed: 789,Unnamed: 790
0,GSS Code,Local Authority,CHN1 2010-11 Real,CHN1 2011-12 Real,CHN1 2012-13 Real,CHN1 2013-14 Real,CHN1 2014-15 Real,CHN1 2015-16 Real,CHN1 2016-17 Real,CHN1 2017-18 Real,...,CLIM2 2010-11,CLIM2 2011-12,CLIM2 2012-13,CLIM2 2013-14,CLIM2 2014-15,CLIM2 2015-16,CLIM2 2016-17,CLIM2 2017-18,CLIM2 2018-19,CLIM2 2019-20
1,S12000033,Aberdeen City,5843.392536,5663.149287,5638.279125,5436.609482,5747.654013,5542.850302,5287.489005,5409.653135,...,7.353521,6.766143,7.032143,6.618359,5.902865,5.653752,5.244611,5.027264,4.975232,dna
2,S12000034,Aberdeenshire,5708.665025,5309.324916,5313.950127,5391.912414,5309.639468,5545.650405,5506.595196,5466.342245,...,8.219167,7.307813,7.533721,7.30263,6.846281,6.516083,6.252694,6.265013,5.99045,dna
3,S12000041,Angus,5508.700818,5416.303192,5191.20614,5069.635562,5169.90078,5192.03655,5339.527096,5324.515572,...,7.511702,6.843333,7.173612,6.933644,6.353447,6.106511,5.801537,5.692337,5.60936,dna
4,S12000035,Argyll & Bute,6889.855421,6527.241093,6564.822198,6521.655789,6236.367875,6111.48398,6159.650534,6940.497861,...,8.316915,7.629353,8.115511,7.595587,7.096742,6.801364,6.429395,6.354077,6.254651,dna
5,S12000005,Clackmannanshire,5209.750091,5066.881518,4642.939951,4344.771817,4272.949411,4386.102558,4811.884361,5084.725548,...,9.697272,8.800269,8.172508,8.132414,6.934098,6.769578,7.425598,7.067078,7.125554,dna
6,S12000006,Dumfries & Galloway,6786.459644,6745.586343,6345.018508,6293.491569,6125.107996,6124.44538,5970.096193,5922.612294,...,9.168621,8.35249,8.571143,8.39446,7.717804,7.515711,7.312967,7.200578,6.986673,dna
7,S12000042,Dundee City,5574.397221,5282.640954,4979.580981,4839.981025,4697.026648,4568.322182,4740.292536,4988.055977,...,6.449618,5.863942,6.058356,5.833258,5.160722,4.97802,4.675949,4.410494,4.352415,dna
8,S12000008,East Ayrshire,5515.882407,5099.185721,5181.318482,5097.955925,5018.281803,5313.893256,5467.442294,5446.227987,...,6.398457,5.808687,5.916354,5.780448,5.27996,5.110176,4.917649,4.847687,4.794713,dna
9,S12000045,East Dunbartonshire,5677.540751,5521.03828,5336.923954,5117.209155,5087.961506,4929.528861,5094.540646,4927.111978,...,5.240444,4.818997,4.831574,4.717572,4.256985,4.05552,3.891443,3.738655,3.693711,dna


In [5]:
# Unpivot all other dataframes so that they are in a more sensible starting point. Each dataframe will require it's own specific transformations so this is as far as the generic steps applied to all dataframes can go.
cashIndicators = unpivotData(cashIndicators)
realIndicators = unpivotData(realIndicators)
cashIndicators_NumeratorDenominator = unpivotData(cashIndicators_NumeratorDenominator)
realIndicators_NumeratorDenominator = unpivotData(realIndicators_NumeratorDenominator)

# Output realIndicators to give an example of the output from this step.
realIndicators


,GSS Code,Local Authority,Attribute,Value
0,S12000033,Aberdeen City,CHN1 2010-11 Real,5843.392536
1,S12000034,Aberdeenshire,CHN1 2010-11 Real,5708.665025
2,S12000041,Angus,CHN1 2010-11 Real,5508.700818
3,S12000035,Argyll & Bute,CHN1 2010-11 Real,6889.855421
4,S12000005,Clackmannanshire,CHN1 2010-11 Real,5209.750091
...,...,...,...,...
26032,S12000029,South Lanarkshire,CLIM2 2019-20,dna
26033,S12000030,Stirling,CLIM2 2019-20,dna
26034,S12000039,West Dunbartonshire,CLIM2 2019-20,dna
26035,S12000040,West Lothian,CLIM2 2019-20,dna


### 4.2 Establish Dataframe from SharePoint Indicator Information List

#### 4.2.1 Get Auth token using client credentials of established SP App for the LGBF site

In [6]:
# Establish data to be passed as part of POST request to obtain session token from SharePoint
data = {
    'grant_type':'client_credentials',
    'resource': "00000003-0000-0ff1-ce00-000000000000/" + SP_tenant + ".sharepoint.com@" + SP_tenantid, 
    'client_id': SP_clientid,
    'client_secret': SP_clientsecret,
}

# Establish headers to be passed as part of POST request to obtain session token from SharePoint
headers = {
    'Content-Type':'application/x-www-form-urlencoded'
}

# Construct url variable for token request
url = "https://accounts.accesscontrol.windows.net/" + SP_tenantid + "/tokens/OAuth/2"

# Send POST request to obtain session token and then load the response into json_data
r = requests.post(url, data=data, headers=headers)
json_data = json.loads(r.text)

#### 4.2.2 Get SharePoint list data for Indicator Information list

In [7]:
# Establish headers to be passed as part of GET request to obtain data from SharePoint list. Using the 'access_token' obtained in the previous step
headers = {
    'Authorization': "Bearer " + json_data['access_token'],
    'Accept':'application/json;odata=verbose',
    "Accept-Charset": "utf-8",
    'Content-Type': 'application/json;odata=verbose'
}

# Construct url variable for data request
url = "https://" + SP_tenant + ".sharepoint.com/sites/PPA/LGBF/_api/web/lists/GetByTitle('Indicator%20Information')/items"

# Send GET request to obtain list data. Extract and normalize this data into a dataframe.
indicatorInformation = requests.get(url, headers=headers)
indicatorInformation = indicatorInformation.json()
indicatorInformation = pd.json_normalize(indicatorInformation['d']['results'])

indicatorInformation

,FileSystemObjectType,Id,ServerRedirectedEmbedUri,ServerRedirectedEmbedUrl,Title,Code,Code_x0028_SortableA_x002d_Z_x00,ReportingPeriod,MeasureType,NumberFormat,...,GetDlpPolicyTip.__deferred.uri,FieldValuesAsHtml.__deferred.uri,FieldValuesAsText.__deferred.uri,FieldValuesForEdit.__deferred.uri,File.__deferred.uri,Folder.__deferred.uri,LikedByInformation.__deferred.uri,ParentList.__deferred.uri,Properties.__deferred.uri,Versions.__deferred.uri
0,0,18,None,,Net cost per Waste collection per premises,ENV1a,ENV 01a,Annual,Cost,'£ '0,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
1,0,19,None,,Net cost per Waste disposal per premises,ENV2a,ENV 02a,Annual,Cost,'£ '0,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
2,0,20,None,,The % of total household waste arising that is...,ENV6,ENV 06,Annual,Percentage,0.0%,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
3,0,21,None,,% of adults satisfied with refuse collection,ENV7a,ENV 07a,3 Year Aggregates,Percentage,0.0%,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
4,0,22,None,,"Net cost of street cleaning per 1,000 population",ENV3a,ENV 03a,Annual,Cost,'£ '0,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0,112,None,,Actual outturn as a percentage of budgeted exp...,FINSUS5,FINSUS 05,Annual,Percentage,0.0%,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://sti

#### 4.2.3 Format Dataframe Appropriately

In [8]:
# Drop system columns not needed in dataframe
indicatorInformation = indicatorInformation.drop(columns = ['FileSystemObjectType','Id','ServerRedirectedEmbedUri','ServerRedirectedEmbedUrl','GetDlpPolicyTip.__deferred.uri','FieldValuesAsHtml.__deferred.uri','FieldValuesAsText.__deferred.uri','FieldValuesForEdit.__deferred.uri','File.__deferred.uri','Folder.__deferred.uri','LikedByInformation.__deferred.uri','ParentList.__deferred.uri','Properties.__deferred.uri','Versions.__deferred.uri','Attachments','GUID','__metadata.id','__metadata.uri','__metadata.etag','__metadata.type','FirstUniqueAncestorSecurableObject.__deferred.uri','RoleAssignments.__deferred.uri','AttachmentFiles.__deferred.uri','AttachmentFiles.__deferred.uri','ContentType.__deferred.uri','ID','Modified','Created','AuthorId','EditorId','OData__UIVersionString','ContentTypeId','ComplianceAssetId'])

# Rename columns appropriately
indicatorInformation = indicatorInformation.rename(columns = {'Code_x0028_SortableA_x002d_Z_x00': 'Code_Sortable', 'Ranking_x0020_Type' : 'Ranking_Type', 'NumberFormat_x0028_notext_x0029_' : "NumberFormat_NoText", "GoldilocksRankingMidpoint_x0028_" : "Ranking_GoldilocksMidpoint"})

# Convert column types to ensure that they can be compared to other data later.
indicatorInformation['Code'] = indicatorInformation['Code'].astype(str)
indicatorInformation['Numerator_Match'] = indicatorInformation['Numerator_Match'].astype(str)
indicatorInformation['Denominator_Match'] = indicatorInformation['Denominator_Match'].astype(str)
indicatorInformation

,Title,Code,Code_Sortable,ReportingPeriod,MeasureType,NumberFormat,YMin,YMax,ISCategory,Committee,...,Ranking_Type,NumberFormat_NoText,Source,Numerator_Correct,Denominator_Correct,Numerator_Match,Denominator_Match,Numerator_Multipier,Denominator_Multiplier,Ranking_GoldilocksMidpoint
0,Net cost per Waste collection per premises,ENV1a,ENV 01a,Annual,Cost,'£ '0,NaN,NaN,Environmental Services,Environment and Housing,...,Ascending,0,For more details on Net Waste Collection Costs...,Waste collection - Net expenditure,Number of Premises for Refuse Collection,Waste collection - Net expenditure (£000s),Number of Premises for Refuse Collection,1000.0,1.000,NaN
1,Net cost per Waste disposal per premises,ENV2a,ENV 02a,Annual,Cost,'£ '0,60.0,NaN,Environmental Services,Environment and Housing,...,Ascending,0,For more details on Net Waste Collection Costs...,Waste disposal - Net expenditure,Number of Premises for Refuse Collection,Waste disposal - Net expenditure (£000s),Number of Premises for Refuse Collection,1000.0,1.000,NaN
2,The % of total household waste arising that is...,ENV6,ENV 06,Annual,Percentage,0.0%,30.0,60.0,Environmental Services,Environment and Housing,...,Descending,0.000,Data is available for each council on the SEPA...,None,None,None,None,NaN,NaN,NaN
3,% of adults satisfied with refuse collection,ENV7a,ENV 07a,3 Year Aggregates,Percentage,0.0%,60.0,100.0,Environmental Services,Environment and Housing,...,Descending,0.000,This data is taken from the Scottish Household...,None,None,None,None,NaN,NaN,NaN
4,"Net cost of street cleaning per 1,000 population",ENV3a,ENV 03a,Annual,Cost,'£ '0,10000.0,NaN,Environmental Services,Environment and Housing,...,Ascending,0,For more details on LFR Costs see LFRs Guidanc...,Street cleaning -net expenditure,MYE Population (Thousands),Street cleaning -net expenditure (£000s),MYE Population,1000.0,0.001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Actual outturn as a percentage of budgeted exp...,FINSUS5,FINSUS 05,Annual,Percentage,0.0%,90.0,110.0,Financial Sustainability,Finance and Economy,...,Descending,0.000,Councils return this data direct to the Improv...,Net outrun revenue expenditure as reported to ...,Net budgeted revenue expenditure as reported t...,Net outrun revenue expenditure as reported to ...,Net budgeted revenue expenditure as reported t...,1000.0,1000.000,1.0
94,CO2 emissions area wide per capita,CLIM1,CLIM 01,Annual,Cost,0.00,4.0,NaN,Tackling Climate Change,Environment and Housing,...,Ascending,0.00,Data for this measure is available from 2010/1...,None,None,None,None,NaN,NaN,NaN
95,CO2 emissions area wide: emissions within scop...,CLIM2,CLIM 02,Annual,Cost,0.00,4.0,NaN,Tackling Climate Change,Environment and Housing,...,Ascending,0.00,Data for this measure is available from 2010/1...,None,None,None,None,NaN,NaN,NaN
96,Cost of Trading Standards and environmental he...,ENV5,ENV 05,Annual,Cost,'£ '0,NaN,NaN,Environmental Services,Environment and Housing,...,Ascending,0,None,Trading Standards & Environmental Health Gross...,MYE Population (Thousands),Trading Standards & Env. Health Gross Expend,MYE Population,1000.0,0.001,NaN


### 4.3 Establish Family Group Dataframe

In [9]:
#Get Family Group Table information from the Improvement Service's website
html = requests.get('https://www.improvementservice.org.uk/benchmarking/how-do-we-compare-councils').content
htmlTables = pd.read_html(html)
FamilyGroups_CSWH = htmlTables[0]
FamilyGroups_ECLEDC = htmlTables[1]

#Unpivot the data in ECLEDC
FamilyGroups_ECLEDC = FamilyGroups_ECLEDC.assign(id= 1)
FamilyGroups_ECLEDC = pd.melt(FamilyGroups_ECLEDC, id_vars=['id'], value_vars=['Family Group 1', 'Family Group 2', 'Family Group 3', 'Family Group 4'],var_name='Family_Group', value_name='Local_Authority')
FamilyGroups_ECLEDC = FamilyGroups_ECLEDC.drop(columns = ['id'])

#The table on the Improvement Service's website does not separate the local authority values into separate table rows but instead puts them all in as one piece of text without a delimeter. In the HTML they are separated by </br> but this does not work with pandas read_html. The only delimeter that can be used is the transition from lower case to uppercase. This is because the only time words run together are between local authorities. Here we replace those transitions with ";" so they can be expanded to new rows.
FamilyGroups = []
Local_Authority = []
Type = []
for row in FamilyGroups_ECLEDC.itertuples() :
    la_list = re.sub(r'(?<=[a-z])(?=[A-Z])', ';', row.Local_Authority)
    Local_Authority.append(la_list)
    FamilyGroups.append(row.Family_Group)
    Type.append('Environmental, Culture & Leisure, Economic Development, Corporate and Property indicators')
    
#Create dataframe to contain the results and rename columns appropriately
FamilyGroups_ECLEDC = pd.DataFrame(Local_Authority)
FamilyGroups_ECLEDC = FamilyGroups_ECLEDC.assign(Type = Type)
FamilyGroups_ECLEDC = FamilyGroups_ECLEDC.assign(Family_Group = FamilyGroups)
FamilyGroups_ECLEDC = FamilyGroups_ECLEDC.rename(columns = {0 : 'Local_Authority'})

#Unpivot the data in CSWH
FamilyGroups_CSWH = FamilyGroups_CSWH.assign(id= 1)
FamilyGroups_CSWH = pd.melt(FamilyGroups_CSWH, id_vars=['id'], value_vars=['Family Group 1', 'Family Group 2', 'Family Group 3', 'Family Group 4'],var_name='Family_Group', value_name='Local_Authority')
FamilyGroups_CSWH = FamilyGroups_CSWH.drop(columns = ['id'])

#The table on the Improvement Service's website does not separate the local authority values into separate table rows but instead puts them all in as one piece of text without a delimeter. In the HTML they are separated by </br> but this does not work with pandas read_html. The only delimeter that can be used is the transition from lower case to uppercase. This is because the only time words run together are between local authorities. Here we replace those transitions with ";" so they can be expanded to new rows.
FamilyGroups = []
Local_Authority = []
Type = []
for row in FamilyGroups_CSWH.itertuples() :
    la_list = re.sub(r'(?<=[a-z])(?=[A-Z])', ';', row.Local_Authority)
    Local_Authority.append(la_list)
    FamilyGroups.append(row.Family_Group)
    Type.append('Children, Social Work and Housing indicators')
    
#Create dataframe to contain the results and rename columns appropriately
FamilyGroups_CSWH = pd.DataFrame(Local_Authority)
FamilyGroups_CSWH = FamilyGroups_CSWH.assign(Type = Type)
FamilyGroups_CSWH = FamilyGroups_CSWH.assign(Family_Group = FamilyGroups)
FamilyGroups_CSWH = FamilyGroups_CSWH.rename(columns = {0 : 'Local_Authority'})


#Concatenate dataframes together and expand the delimeted local authorities to new rows.
Family_Groups = pd.concat([FamilyGroups_ECLEDC, FamilyGroups_CSWH])
Family_Groups = Family_Groups.assign(Local_Authority=Family_Groups.Local_Authority.str.split(";")).explode('Local_Authority')
#There is a mismatch between the naming of Edinburgh on the Improvement Services Family Groupings web page and the naming in the raw data file. Replace the text here to allow merges with the raw data file later.
Family_Groups['Local_Authority'] = Family_Groups['Local_Authority'].str.replace('Edinburgh, City of','Edinburgh City')
Family_Groups

,Local_Authority,Type,Family_Group
0,Eilean Siar,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
0,Argyll & Bute,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
0,Shetland Islands,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
0,Highland,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
0,Orkney Islands,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
...,...,...,...
3,North Ayrshire,"Children, Social Work and Housing indicators",Family Group 4
3,North Lanarkshire,"Children, Social Work and Housing indicators",Family Group 4
3,Inverclyde,"Children, Social Work and Housing indicators",Family Group 4
3,West Dunbartonshire,"Children, Social Work and Housing indicators",Family Group 4


## 5. Transform Each Dataframe Containing Indicator Data and Merge DataFrames Into One Master DataFrame
Apply relevant transformation steps to each of the Dataframes below and join them into a single fact table containing Cash Values, Real Values, Cash Numerators, Cash Denominators, Real Numerators and Real Denominators.

### 5.1 Transform realIndicators Dataframe

In [10]:
#Import the realIndicators into what will become the final table
all_LGBFData = realIndicators
#Split the Attribute column into its parts using space as a delimeter
all_LGBFData[['Code','Period','Other']] = all_LGBFData['Attribute'].str.split(" ", n=2, expand=True)
#Remove the Attribute column and the redundant other column, rename columns to avoid spaces and to identify the value type as we intend to merge all dataframes into a single dataframe.
all_LGBFData = all_LGBFData.drop(columns = ['Attribute','Other'])
all_LGBFData = all_LGBFData.rename(columns = {'Value': 'Real_Value', 'Local Authority' : 'Local_Authority'})
# Save Scotland values only for merge with averages table later
ISScottishValues_Real = all_LGBFData[(all_LGBFData.Local_Authority == "Scotland")]
#Remove all of the "Scottish Average" Rows as we intend to provide both types of average later in order to provide a more "Complete" data set. Reset the index after this to ensure it is consistent.
all_LGBFData = all_LGBFData[(all_LGBFData.Local_Authority != "Scotland")]
all_LGBFData = all_LGBFData.reset_index(drop=True)
all_LGBFData

,GSS Code,Local_Authority,Real_Value,Code,Period
0,S12000033,Aberdeen City,5843.392536,CHN1,2010-11
1,S12000034,Aberdeenshire,5708.665025,CHN1,2010-11
2,S12000041,Angus,5508.700818,CHN1,2010-11
3,S12000035,Argyll & Bute,6889.855421,CHN1,2010-11
4,S12000005,Clackmannanshire,5209.750091,CHN1,2010-11
...,...,...,...,...,...
25243,S12000028,South Ayrshire,dna,CLIM2,2019-20
25244,S12000029,South Lanarkshire,dna,CLIM2,2019-20
25245,S12000030,Stirling,dna,CLIM2,2019-20
25246,S12000039,West Dunbartonshire,dna,CLIM2,2019-20


### 5.2 Transform cashIndicators Dataframe

In [11]:
#Split the Attribute column into its parts using space as a delimeter
cashIndicators[['Code','Period']] = cashIndicators['Attribute'].str.split(" ", n=2, expand=True)
#Remove the Attribute column and the redundant other column, rename columns to avoid spaces and to identify the value type as we intend merge into a single dataframe.
cashIndicators = cashIndicators.drop(columns = ['Attribute'])
cashIndicators = cashIndicators.rename(columns = {'Value': 'Cash_Value', 'Local Authority' : 'Local_Authority'})
#Merge with saved scotland real values to merge with Scottish Averages Dataframe later
ISScottishValues_Cash = cashIndicators[(cashIndicators.Local_Authority == "Scotland")]
ISScottishValues = ISScottishValues_Real.merge(ISScottishValues_Cash, how = 'left', on = ['Local_Authority','Code','Period','GSS Code'], suffixes = ('_Real','_Cash'))
ISScottishValues = ISScottishValues.drop(columns = ['GSS Code','Local_Authority'])
#Remove all of the "Scottish Average" Rows as we intend to provide both types of average later in order to provide a more "Complete" data set. Reset the index after this to ensure it is consistent.
cashIndicators = cashIndicators[(cashIndicators.Local_Authority != "Scotland")]
cashIndicators = cashIndicators.reset_index(drop=True)
cashIndicators

,GSS Code,Local_Authority,Cash_Value,Code,Period
0,S12000033,Aberdeen City,4960.336134,CHN1,2010-11
1,S12000034,Aberdeenshire,4845.968712,CHN1,2010-11
2,S12000041,Angus,4676.223196,CHN1,2010-11
3,S12000035,Argyll & Bute,5848.657025,CHN1,2010-11
4,S12000005,Clackmannanshire,4422.450053,CHN1,2010-11
...,...,...,...,...,...
25243,S12000028,South Ayrshire,dna,CLIM2,2019-20
25244,S12000029,South Lanarkshire,dna,CLIM2,2019-20
25245,S12000030,Stirling,dna,CLIM2,2019-20
25246,S12000039,West Dunbartonshire,dna,CLIM2,2019-20


### 5.4 Transform NumeratorDenominator Dataframes

In [12]:
# Create a dictionary that can be searched independant of case from the indicatorInformation dataframe. This avoids having to loop the dataframe and exponentially speeds this step up.
indicatorInformation_dict = indicatorInformation.set_index('Code').to_dict('index')
indicatorInformation_dict = NocaseDict(indicatorInformation_dict)
# Display example dictionary item for reference
indicatorInformation_dict['env1a']

{'Title': 'Net cost per Waste collection per premises',
 'Code_Sortable': 'ENV 01a',
 'ReportingPeriod': 'Annual',
 'MeasureType': 'Cost',
 'NumberFormat': "'£ '0",
 'YMin': nan,
 'YMax': nan,
 'ISCategory': 'Environmental Services',
 'Committee': 'Environment and Housing',
 'FamilyGrouping': 'Environmental, Culture & Leisure, Economic Development, Corporate and Property indicators',
 'StirlingService': 'Environment & Place ; Waste Services',
 'Ranking_Type': 'Ascending',
 'NumberFormat_NoText': '0',
 'Source': 'For more details on Net Waste Collection Costs see LFRs Guidance notes for waste collection services:\nhttp://www.scotland.gov.uk/Topics/Statistics/Browse/Local-Government-Finance/ReturnLFR/LFR1112Guidance (LFR 06, Waste Collection)\nData for Number of Premises can be found on the Audit Scotland website:\nhttp://www.audit-scotland.gov.uk/performance/service/ Waste Management\nFrom 2013/14 Councils return the number of premises data direct to the Improvement Service. Data is ava

In [13]:
# Define function to convert numerator/denominator sheets into column based dataframes that only contain the necessary columns

Types = []
Codes = []
Periods = []
Values = []
PreviousType = None
PreviousAttribute = None

def ConvertNumDenDF(df,prefix) :
    #!!! This is a bit hacky - Requires proper fix in unpivot data table!!!
    df = df[pd.notnull(df['Attribute'])]
    df = df.rename(columns = {'Local Authority' : 'Local_Authority'})

    # Filter the dataframe to remove Scotland values as they will be calculated later.
    df = df[(df.Local_Authority != "Scotland")]

    #Get first word from Attribute column (this will be the code where the value specified is not a numerator or denominator)
    df['Code'] = df['Attribute'].str.split(' ').str[0]
    df['Year'] = df['Attribute'].str.split(' ').str[1]

    df['Value'] = df['Value'].apply(pd.to_numeric,errors='coerce')

    #Loop dataframe and establish Value Type, Period and Code based on positioning and checking the code column in the indicatorInformation dataframe. This will have issues if new indicators are introduced and the codes have not yet been added to the Indicator Information SharePoint List. Manual checking required as not sure how to protect against this.
    Types = []
    Codes = []
    Periods = []
    Values = []
    PreviousType = None
    PreviousAttribute = None

    for row in df.itertuples():
        
        if row.Attribute == PreviousAttribute:
            Types.append(PreviousType)
            Codes.append(LastCode)
            Periods.append(LastPeriod)
            PreviousAttribute = row.Attribute

        elif any(row.Code in string for string in indicatorInformation['Code']):
            Types.append("Value")
            LastCode = row.Code
            Codes.append(LastCode)
            LastPeriod = row.Year
            Periods.append(LastPeriod)
            PreviousType = "Value"
            PreviousAttribute = row.Attribute
            Numerator_Multiplier = indicatorInformation_dict.get(row.Code).get('Numerator_Multipier')
            Denominator_Multiplier = indicatorInformation_dict.get(row.Code).get('Denominator_Multiplier')

        elif PreviousType == "Value" :
            Types.append("Numerator")
            Codes.append(LastCode)
            Periods.append(LastPeriod)
            PreviousType = "Numerator"
            PreviousAttribute = row.Attribute

        elif PreviousType == "Numerator":
            Types.append("Denominator")
            Codes.append(LastCode)
            Periods.append(LastPeriod)
            PreviousType = "Denominator"
            PreviousAttribute = row.Attribute

        else:
            Types.append(None)
            Codes.append(None)
            Periods.append(None)
            PreviousAttribute = row.Attribute


        if PreviousType == "Value":
            Values.append(row.Value)
        elif PreviousType == "Numerator":
            Values.append(row.Value * Numerator_Multiplier)
        elif PreviousType == "Denominator":
            Values.append(row.Value * Denominator_Multiplier)

    df['Code'] = Codes
    df['Period'] = Periods
    df['Type'] = Types
    df['Value'] = Values

    df = df[['GSS Code','Local_Authority','Code','Period','Type','Value']]
    df = df[pd.notnull(df['Value'])]

    # Establish new dataframe that contains only the values from realIndicators_NumeratorDenominator that are numerators. Rename the Value column appropriately
    df_Numerators = df[(df.Type == "Numerator")]
    df_Numerators = df_Numerators.rename(columns = {'Value' : prefix + '_Numerator'})
    df_Numerators = df_Numerators.drop('Type', axis=1)

    # Establish new dataframe that contains only the values from realIndicators_NumeratorDenominator that are denominators. Rename the Value column appropriately
    df_Denominators = df[(df.Type == "Denominator")]
    df_Denominators = df_Denominators.rename(columns={'Value': prefix + '_Denominator'})
    df_Denominators = df_Denominators.drop('Type', axis=1)

    # Merge realIndicators_Numerators and realIndicators_Denominators overwriting the previous dataframe realIndicators_NumeratorDenominator so we have a dataframe of the correct length with two value columns. One for numerator and one for denominator.
    df = df_Numerators.merge(df_Denominators, how='left', on=['Local_Authority', 'Code', 'Period', 'GSS Code'], suffixes=('_Num', '_Den'))
    return df

In [14]:
# Convert sheets and output one as an example
realIndicators_NumeratorDenominator = ConvertNumDenDF(realIndicators_NumeratorDenominator,'Real')
cashIndicators_NumeratorDenominator = ConvertNumDenDF(cashIndicators_NumeratorDenominator,'Cash')
cashIndicators_NumeratorDenominator

,GSS Code,Local_Authority,Code,Period,Cash_Numerator,Cash_Denominator
0,S12000033,Aberdeen City,CHN1,2010-11,59028000.0,11900.0
1,S12000034,Aberdeenshire,CHN1,2010-11,92621000.0,19113.0
2,S12000041,Angus,CHN1,2010-11,39472000.0,8441.0
3,S12000035,Argyll & Bute,CHN1,2010-11,33969000.0,5808.0
4,S12000005,Clackmannanshire,CHN1,2010-11,16823000.0,3804.0
...,...,...,...,...,...,...
11576,S12000028,South Ayrshire,FINSUS5,2019-20,261056000.0,267124000.0
11577,S12000029,South Lanarkshire,FINSUS5,2019-20,725520000.0,728541000.0
11578,S12000030,Stirling,FINSUS5,2019-20,219056000.0,218811000.0
11579,S12000039,West Dunbartonshire,FINSUS5,2019-20,222847000.0,223433000.0


### 5.5 Merge all Dataframes into all_LGBFData

In [15]:
# Merge the dataframes together into all_LGBFData. Reorder columns after merge.
all_LGBFData = all_LGBFData.merge(cashIndicators, how = 'left', on = ['Local_Authority','Code','Period','GSS Code'], suffixes = ('_Real','_Cash'))
all_LGBFData = all_LGBFData.merge(realIndicators_NumeratorDenominator, how = 'left', on = ['Local_Authority','Code','Period','GSS Code'], suffixes = ('_all','_realNumDen'))
all_LGBFData = all_LGBFData.merge(cashIndicators_NumeratorDenominator, how = 'left', on = ['Local_Authority','Code','Period','GSS Code'], suffixes = ('_all','_cashNumDen'))
all_LGBFData = all_LGBFData[['GSS Code','Local_Authority','Code','Period','Real_Value','Real_Numerator','Real_Denominator','Cash_Value','Cash_Numerator','Cash_Denominator']]

# Convert value columns to numeric only using errors = 'coerce' to force any text values (dna, *, N/A etc.) to return NaN.
all_LGBFData[['Real_Value','Cash_Value']] = all_LGBFData[['Real_Value','Cash_Value']].apply(pd.to_numeric,errors='coerce')

# Remove rows which contain null or NaN in the Real_Value column as they are non-datapoints and could interfere with average, ranking and quartile calculations later.
all_LGBFData = all_LGBFData[pd.notnull(all_LGBFData['Real_Value'])]

# Convert Percentages to proper decimal percentages
all_LGBFData = all_LGBFData.merge(indicatorInformation[['Code','MeasureType']], how = 'left', on = ['Code'], suffixes = ('_all','_info'))

def ConvertPercent_Real(df) : 
    if df['MeasureType'] == "Percentage":
        return df['Real_Value']/100
    else :
        return df['Real_Value']

def ConvertPercent_Cash(df) : 
    if df['MeasureType'] == "Percentage":
        return df['Cash_Value']/100
    else :
        return df['Cash_Value']

all_LGBFData['Real_Value'] = all_LGBFData.apply(ConvertPercent_Real, axis = 1)
all_LGBFData['Cash_Value'] = all_LGBFData.apply(ConvertPercent_Cash, axis = 1)

all_LGBFData = all_LGBFData.drop(columns = ['MeasureType'])

all_LGBFData

,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator
0,S12000033,Aberdeen City,CHN1,2010-11,5843.392536,6.953637e+07,11900.0,4960.336134,59028000.0,11900.0
1,S12000034,Aberdeenshire,CHN1,2010-11,5708.665025,1.091097e+08,19113.0,4845.968712,92621000.0,19113.0
2,S12000041,Angus,CHN1,2010-11,5508.700818,4.649894e+07,8441.0,4676.223196,39472000.0,8441.0
3,S12000035,Argyll & Bute,CHN1,2010-11,6889.855421,4.001628e+07,5808.0,5848.657025,33969000.0,5808.0
4,S12000005,Clackmannanshire,CHN1,2010-11,5209.750091,1.981789e+07,3804.0,4422.450053,16823000.0,3804.0
...,...,...,...,...,...,...,...,...,...,...
24536,S12000028,South Ayrshire,CLIM2,2018-19,5.700050,NaN,NaN,5.700050,NaN,NaN
24537,S12000029,South Lanarkshire,CLIM2,2018-19,3.970788,NaN,NaN,3.970788,NaN,NaN
24538,S12000030,Stirling,CLIM2,2018-19,6.437136,NaN,NaN,6.437136,NaN,NaN
24539,S12000039,West Dunbartonshire,CLIM2,2018-19,4.201782,NaN,NaN,4.201782,NaN,NaN


## 6. Calculate Averages
There are two types of averages that will be applied to both real and cash values within both the Scottish 32 Councils whole group and the Family Groups for each indicator and period. All averages to be added are as below along with the column names to be assigned.

1. Scottish Averages
    1. Average of Local Authority Real Values - ScotAv_LA_Real
    2. Average of Local Authority Cash Values - ScotAv_LA_Cash
    3. Sum of Real Numerators Divided by Sum of Real Denominators - ScotAv_NumDen_Real
    4. Sum of Cash Numerators Divided by Sum of Cash Denominators - ScotAv_NumDen_Cash
    5. Average of Local Authority Real Numerator Values - ScotAv_LA_Num_Real
    6. Average of Local Authority Real Denominator Values - ScotAv_LA_Den_Real
    5. Average of Local Authority Cash Numerator Values - ScotAv_LA_Num_Cash
    6. Average of Local Authority Cash Denominator Values - ScotAv_LA_Den_Cash
2. Family Group Averages
    1. Average of Local Authority Real Values - FamilyAv_LA_Real
    2. Average of Local Authority Cash Values - FamilyAv_LA_Cash
    3. Sum of Real Numerators Divided by Sum of Real Denominators - FamilyAv_NumDen_Real
    4. Sum of Cash Numerators Divided by Sum of Cash Denominators - FamilyAv_NumDen_Cash
    5. Average of Local Authority Real Numerator Values - FamilyAv_LA_Num_Real
    6. Average of Local Authority Real Denominator Values - FamilyAv_LA_Den_Real
    5. Average of Local Authority Cash Numerator Values - FamilyAv_LA_Num_Cash
    6. Average of Local Authority Cash Denominator Values - FamilyAv_LA_Den_Cash

### 6.1 Scottish Averages

In [16]:
#Sum then divide to get the average resulting from sum of numerator values divided by sum of denominator values for all Real Values.
real_NumeratorDenominatorAverages = realIndicators_NumeratorDenominator.groupby(['Code','Period'], as_index = False).sum()
real_NumeratorDenominatorAverages['ScotAv_NumDen_Real'] = real_NumeratorDenominatorAverages['Real_Numerator'] / real_NumeratorDenominatorAverages['Real_Denominator']
real_NumeratorDenominatorAverages = real_NumeratorDenominatorAverages[['Code','Period','ScotAv_NumDen_Real']]
real_NumeratorDenominatorAverages = real_NumeratorDenominatorAverages.replace([np.inf, -np.inf], np.nan)

#Sum then divide to get the average resulting from sum of numerator values divided by sum of denominator values for all Cash Values.
cash_NumeratorDenominatorAverages = cashIndicators_NumeratorDenominator.groupby(['Code','Period'], as_index = False).sum()
cash_NumeratorDenominatorAverages['ScotAv_NumDen_Cash'] = cash_NumeratorDenominatorAverages['Cash_Numerator'] / cash_NumeratorDenominatorAverages['Cash_Denominator']
cash_NumeratorDenominatorAverages = cash_NumeratorDenominatorAverages[['Code','Period','ScotAv_NumDen_Cash']]
cash_NumeratorDenominatorAverages = cash_NumeratorDenominatorAverages.replace([np.inf, -np.inf], np.nan)

#Calculate mean averages for all other columns and then merge dataframes together to finalise dataframe
ScottishAverages_LA = all_LGBFData.groupby(['Code','Period'], as_index = False).mean()
ScottishAverages =  ScottishAverages_LA.merge(real_NumeratorDenominatorAverages, how = 'left', on = ['Code','Period'], suffixes = ('_all','_RealNumDen'))
ScottishAverages =  ScottishAverages.merge(cash_NumeratorDenominatorAverages, how = 'left', on = ['Code','Period'], suffixes = ('_all','_CashNumDen'))

# Reorder and rename columns appropriately
ScottishAverages = ScottishAverages.rename(columns = {'Real_Value': 'ScotAv_LA_Real', 'Cash_Value' : 'ScotAv_LA_Cash', 'Real_Numerator' : 'ScotAv_LA_Num_Real', 'Real_Denominator' : 'ScotAv_LA_Den_Real','Cash_Numerator' : 'ScotAv_LA_Num_Cash', 'Cash_Denominator' : 'ScotAv_LA_Den_Cash'})

ScottishAverages = ScottishAverages.merge(indicatorInformation[['Code','Code_Sortable']], how = 'left', on = ['Code'], suffixes = ('_ScotAv','_info'))
ScottishAverages = ScottishAverages[['Code_Sortable','Period','ScotAv_LA_Real','ScotAv_LA_Num_Real','ScotAv_LA_Den_Real','ScotAv_NumDen_Real','ScotAv_LA_Cash','ScotAv_LA_Num_Cash','ScotAv_LA_Den_Cash','ScotAv_NumDen_Cash']]

ScottishAverages = ScottishAverages.rename(columns = {'Code_Sortable':'Code'})

# Add Relationship Key Column
ScottishAverages['Key_CodePeriod'] = ScottishAverages['Code'] + ScottishAverages['Period']

ScottishAverages

,Code,Period,ScotAv_LA_Real,ScotAv_LA_Num_Real,ScotAv_LA_Den_Real,ScotAv_NumDen_Real,ScotAv_LA_Cash,ScotAv_LA_Num_Cash,ScotAv_LA_Den_Cash,ScotAv_NumDen_Cash,Key_CodePeriod
0,C&L 01,2010-11,3.884901,5.943202e+06,1.420619e+06,4.183529,3.297813,5.045062e+06,1.420619e+06,3.551312,C&L 012010-11
1,C&L 01,2011-12,3.637954,5.639688e+06,1.506323e+06,3.744009,3.135006,4.860000e+06,1.506323e+06,3.226399,C&L 012011-12
2,C&L 01,2012-13,3.489651,5.757252e+06,1.613272e+06,3.568681,3.068692,5.062750e+06,1.613272e+06,3.138188,C&L 012012-13
3,C&L 01,2013-14,3.436652,5.936639e+06,1.666276e+06,3.562818,3.076593,5.314656e+06,1.666276e+06,3.189541,C&L 012013-14
4,C&L 01,2014-15,3.316311,5.377523e+06,1.647039e+06,3.264963,3.009798,4.880500e+06,1.647039e+06,2.963196,C&L 012014-15
...,...,...,...,...,...,...,...,...,...,...,...
778,SW 08,2015-16,822.315884,1.251138e+04,1.367866e+01,914.664041,822.315884,1.251138e+04,1.367866e+01,914.664041,SW 082015-16
779,SW 08,2016-17,794.060450,1.160719e+04,1.382216e+01,839.752300,794.060450,1.160719e+04,1.382216e+01,839.752300,SW 082016-17
780,SW 08,2017-18,708.904317,1.066372e+04,1.399909e+01,761.743506,708.904317,1.066372e+04,1.399909e+01,761.743506,SW 082017-18
781,SW 08,2018-19,737.560000,1.126409e+04,1.421050e+01,792.659917,737.560000,1.126409e+04,1.421050e+01,792.659917,SW 082018-19


### 6.2 Family Group Averages

In [17]:
#Merge all_LGBFData with indicatorInformation and Family_Groups to add the additional grouping information that enables Family Group average calculations.
FamilyAverages_LA = all_LGBFData.merge(indicatorInformation[['Code','FamilyGrouping']], how = 'left', on = ['Code'], suffixes = ('_all','_info'))
FamilyAverages_LA = FamilyAverages_LA.rename(columns = {'FamilyGrouping' : 'Type'})
FamilyAverages_LA = FamilyAverages_LA.merge(Family_Groups,how ='left', on = ['Local_Authority','Type'], suffixes = ('_all','_group'))

#Sum then divide to get the average resulting from sum of numerator values divided by sum of denominator values for all Cash Values.
real_NumeratorDenominatorAverages = FamilyAverages_LA.groupby(['Code','Period','Family_Group'], as_index = False).sum()
real_NumeratorDenominatorAverages['FamilyAv_NumDen_Real'] = real_NumeratorDenominatorAverages['Real_Numerator'] / real_NumeratorDenominatorAverages['Real_Denominator']
real_NumeratorDenominatorAverages = real_NumeratorDenominatorAverages[['Code','Period','Family_Group','FamilyAv_NumDen_Real']]
real_NumeratorDenominatorAverages = real_NumeratorDenominatorAverages.replace([np.inf, -np.inf], np.nan)

#Sum then divide to get the average resulting from sum of numerator values divided by sum of denominator values for all Cash Values.
cash_NumeratorDenominatorAverages = FamilyAverages_LA.groupby(['Code','Period','Family_Group'], as_index = False).sum()
cash_NumeratorDenominatorAverages['FamilyAv_NumDen_Cash'] = cash_NumeratorDenominatorAverages['Cash_Numerator'] / cash_NumeratorDenominatorAverages['Cash_Denominator']
cash_NumeratorDenominatorAverages = cash_NumeratorDenominatorAverages[['Code','Period','Family_Group','FamilyAv_NumDen_Cash']]
cash_NumeratorDenominatorAverages = cash_NumeratorDenominatorAverages.replace([np.inf, -np.inf], np.nan)

#Calculate mean averages for all other columns and then merge dataframes together to finalise dataframe
FamilyAverages_LA = FamilyAverages_LA.groupby(['Code','Period','Family_Group'], as_index = False).mean()
FamilyAverages =  FamilyAverages_LA.merge(real_NumeratorDenominatorAverages, how = 'left', on = ['Code','Period','Family_Group'], suffixes = ('_all','_RealNumDen'))
FamilyAverages =  FamilyAverages.merge(cash_NumeratorDenominatorAverages, how = 'left', on = ['Code','Period','Family_Group'], suffixes = ('_all','_CashNumDen'))

# Reorder and rename columns appropriately
FamilyAverages = FamilyAverages.rename(columns = {'Real_Value': 'FamilyAv_LA_Real', 'Cash_Value' : 'FamilyAv_LA_Cash', 'Real_Numerator' : 'FamilyAv_LA_Num_Real', 'Real_Denominator' : 'FamilyAv_LA_Den_Real','Cash_Numerator' : 'FamilyAv_LA_Num_Cash', 'Cash_Denominator' : 'FamilyAv_LA_Den_Cash'})

FamilyAverages = FamilyAverages.merge(indicatorInformation[['Code','Code_Sortable']], how = 'left', on = ['Code'], suffixes = ('_ScotAv','_info'))
FamilyAverages = FamilyAverages[['Code_Sortable','Period','Family_Group','FamilyAv_LA_Real','FamilyAv_LA_Num_Real','FamilyAv_LA_Den_Real','FamilyAv_NumDen_Real','FamilyAv_LA_Cash','FamilyAv_LA_Num_Cash','FamilyAv_LA_Den_Cash','FamilyAv_NumDen_Cash']]

FamilyAverages = FamilyAverages.rename(columns = {'Code_Sortable':'Code'})

# Add Relationship Key Column
FamilyAverages['Key_CodePeriodFamily_Group'] = FamilyAverages['Code'] + FamilyAverages['Period'] + FamilyAverages['Family_Group']

FamilyAverages

,Code,Period,Family_Group,FamilyAv_LA_Real,FamilyAv_LA_Num_Real,FamilyAv_LA_Den_Real,FamilyAv_NumDen_Real,FamilyAv_LA_Cash,FamilyAv_LA_Num_Cash,FamilyAv_LA_Den_Cash,FamilyAv_NumDen_Cash,Key_CodePeriodFamily_Group
0,C&L 01,2010-11,Family Group 1,3.547323,3.623748e+06,9.231241e+05,3.925526,3.011250,3076125.000,9.231241e+05,3.332298,C&L 012010-11Family Group 1
1,C&L 01,2010-11,Family Group 2,4.249720,4.914420e+06,1.165654e+06,4.216020,3.607500,4171750.000,1.165654e+06,3.578893,C&L 012010-11Family Group 2
2,C&L 01,2010-11,Family Group 3,3.645983,5.050776e+06,1.393280e+06,3.625096,3.095000,4287500.000,1.393280e+06,3.077270,C&L 012010-11Family Group 3
3,C&L 01,2010-11,Family Group 4,4.096577,1.018387e+07,2.200419e+06,4.628149,3.477500,8644875.000,2.200419e+06,3.928741,C&L 012010-11Family Group 4
4,C&L 01,2011-12,Family Group 1,3.526728,3.728896e+06,1.036476e+06,3.597667,3.039157,3213375.000,1.036476e+06,3.100289,C&L 012011-12Family Group 1
...,...,...,...,...,...,...,...,...,...,...,...,...
3127,SW 08,2018-19,Family Group 4,695.967500,9.248375e+03,1.452962e+01,636.518492,695.967500,9248.375,1.452962e+01,636.518492,SW 082018-19Family Group 4
3128,SW 08,2019-20,Family Group 1,528.139812,9.926125e+03,1.456925e+01,681.306519,528.139812,9926.125,1.456925e+01,681.306519,SW 082019-20Family Group 1
3129,SW 08,2019-20,Family Group 2,704.286174,8.919500e+03,1.168712e+01,763.190263,704.286174,8919.500,1.168712e+01,763.190263,SW 082019-20Family Group 2
3130,SW 08,2019-20,Family Group 3,875.456281,1.561462e+04,1.722200e+01,906.667344,875.456281,15614.625,1.722200e+01,906.667344,SW 082019-20Family Group 3


## 7 Calculate Ranks & Percentiles

### 7.1 Scottish Ranks & Percentiles

In [18]:
#Copy all_LGBFData into ScottishRanks to avoid any steps affecting the all_LGBFData dataframe.
ScottishRanks = all_LGBFData.copy(deep=True)

#Add columns for both ascending and descending ranks and percentiles. The correct versions will be chosen later.
ScottishRanks['ScotRank_Desc'] = all_LGBFData.groupby(['Code','Period'])['Real_Value'].rank('dense',ascending = False).astype(int)
ScottishRanks['ScotRank_Asc'] = all_LGBFData.groupby(['Code','Period'])['Real_Value'].rank('dense',ascending = True).astype(int)
ScottishRanks['ScotRank_Desc_Pct'] = all_LGBFData.groupby(['Code','Period'])['Real_Value'].rank('dense',ascending = False, pct = True).astype(float)
ScottishRanks['ScotRank_Asc_Pct'] = all_LGBFData.groupby(['Code','Period'])['Real_Value'].rank('dense',ascending = True, pct = True).astype(float)
ScottishRanks = ScottishRanks.merge(indicatorInformation[['Code','Ranking_Type']], how = 'left', on = ['Code'], suffixes = ('_ScotAv','_Info'))


#Merge all_LGBFData with indicatorInformation to get the goldilocks mid points for the few indicators that use them.
GoldilocksScottishRanks = all_LGBFData.merge(indicatorInformation[['Code','Ranking_GoldilocksMidpoint']], how = 'left', on = ['Code'], suffixes = ('_Goldi','_Info'))

#Define function that returns unsigned difference between the real values and the goldilocks mid point. This uses the distance function established at the start of the notebook and then absolutes the values returned from it.
def DifferenceFromGoldilocksMidPoint(df) : 
    if df['Ranking_GoldilocksMidpoint'] == None :
        return None
    else :
        return abs(distance(df['Real_Value'],df['Ranking_GoldilocksMidpoint']))

#Calculate ranking and percentile based on distance from goldilocks mid point.
GoldilocksScottishRanks['AbsoluteDifferenceFromGoldilocksMidPoint'] = GoldilocksScottishRanks.apply(DifferenceFromGoldilocksMidPoint, axis = 1)
GoldilocksScottishRanks = GoldilocksScottishRanks[pd.notnull(GoldilocksScottishRanks['AbsoluteDifferenceFromGoldilocksMidPoint'])]
GoldilocksScottishRanks['ScotRank_Goldi'] = GoldilocksScottishRanks.groupby(['Code','Period'])['AbsoluteDifferenceFromGoldilocksMidPoint'].rank('dense',ascending = True).astype(int)
GoldilocksScottishRanks['ScotRank_Goldi_Pct'] = GoldilocksScottishRanks.groupby(['Code','Period'])['AbsoluteDifferenceFromGoldilocksMidPoint'].rank('dense',ascending = True,pct = True).astype(float)

#Merge goldilocks ranking and percentile into ScottishRanks dataframe
ScottishRanks = ScottishRanks.merge(GoldilocksScottishRanks[['Code','Period','Local_Authority','ScotRank_Goldi','ScotRank_Goldi_Pct']], how = 'left', on = ['Code','Period','Local_Authority'], suffixes = ('_ScotRank','_Goldi'))

# Define functions needed to select correct ranking type and percentile type
def ScotRank_select(df) : 
    if df['Ranking_Type'] == "Ascending" :
        return df['ScotRank_Asc']
    elif df['Ranking_Type'] == "Descending" :
        return df['ScotRank_Desc']
    elif df['Ranking_Type'] == "Goldilocks" :
        return df['ScotRank_Goldi']
    else :
        return None

def ScotRank_Pct_select(df) : 
    if df['Ranking_Type'] == "Ascending" :
        return df['ScotRank_Asc_Pct']
    elif df['Ranking_Type'] == "Descending" :
        return df['ScotRank_Desc_Pct']
    elif df['Ranking_Type'] == "Goldilocks" :
        return df['ScotRank_Goldi_Pct']
    else :
        return None

# Apply functions above to create two new columns that contain the correct rank and percentile for each row
ScottishRanks['ScotRank'] = ScottishRanks.apply(ScotRank_select, axis = 1)
ScottishRanks['ScotPct'] = ScottishRanks.apply(ScotRank_Pct_select, axis = 1)

# Add Scottish Ranks to the main fact table
all_LGBFData = ScottishRanks[['GSS Code','Local_Authority','Code','Period','Real_Value','Real_Numerator','Real_Denominator','Cash_Value','Cash_Numerator','Cash_Denominator','ScotRank','ScotPct']]
all_LGBFData

,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct
0,S12000033,Aberdeen City,CHN1,2010-11,5843.392536,6.953637e+07,11900.0,4960.336134,59028000.0,11900.0,22.0,0.68750
1,S12000034,Aberdeenshire,CHN1,2010-11,5708.665025,1.091097e+08,19113.0,4845.968712,92621000.0,19113.0,20.0,0.62500
2,S12000041,Angus,CHN1,2010-11,5508.700818,4.649894e+07,8441.0,4676.223196,39472000.0,8441.0,11.0,0.34375
3,S12000035,Argyll & Bute,CHN1,2010-11,6889.855421,4.001628e+07,5808.0,5848.657025,33969000.0,5808.0,29.0,0.90625
4,S12000005,Clackmannanshire,CHN1,2010-11,5209.750091,1.981789e+07,3804.0,4422.450053,16823000.0,3804.0,4.0,0.12500
...,...,...,...,...,...,...,...,...,...,...,...,...
24536,S12000028,South Ayrshire,CLIM2,2018-19,5.700050,NaN,NaN,5.700050,NaN,NaN,19.0,0.59375
24537,S12000029,South Lanarkshire,CLIM2,2018-19,3.970788,NaN,NaN,3.970788,NaN,NaN,4.0,0.12500
24538,S12000030,Stirling,CLIM2,2018-19,6.437136,NaN,NaN,6.437136,NaN,NaN,25.0,0.78125
24539,S12000039,West Dunbartonshire,CLIM2,2018-19,4.201782,NaN,NaN,4.201782,NaN,NaN,7.0,0.21875


### 7.2 Family Group Ranks & Percentiles

In [19]:
#Merge all_LGBFData with additional data from indicatorInformation and Family_Groups dataframes so that the dataframe contains all of the fields required to group for the purposes of ranking.
FamilyRanks_ForGroup = all_LGBFData.merge(indicatorInformation[['Code','FamilyGrouping']], how = 'left', on = ['Code'], suffixes = ('_all','_info'))
FamilyRanks_ForGroup = FamilyRanks_ForGroup.rename(columns = {'FamilyGrouping' : 'Type'})
FamilyRanks_ForGroup = FamilyRanks_ForGroup.merge(Family_Groups,how ='left', on = ['Local_Authority','Type'], suffixes = ('_all','_group'))

#Copy the dataframe to a new variable. THe FamilyRanks_ForGroup dataframe will be used again later to calculate goldilocks ranks and quartiles.
FamilyRanks = FamilyRanks_ForGroup.copy(deep = True)

#Add columns for both ascending and descending ranks and percentiles. The correct versions will be chosen later.
FamilyRanks['FamilyRank_Desc'] = FamilyRanks_ForGroup.groupby(['Code','Period','Family_Group'])['Real_Value'].rank('dense',ascending = False).astype(int)
FamilyRanks['FamilyRank_Asc'] = FamilyRanks_ForGroup.groupby(['Code','Period','Family_Group'])['Real_Value'].rank('dense',ascending = True).astype(int)
FamilyRanks['FamilyRank_Desc_Pct'] = FamilyRanks_ForGroup.groupby(['Code','Period','Family_Group'])['Real_Value'].rank('dense',ascending = False, pct = True).astype(float)
FamilyRanks['FamilyRank_Asc_Pct'] = FamilyRanks_ForGroup.groupby(['Code','Period','Family_Group'])['Real_Value'].rank('dense',ascending = True, pct = True).astype(float)

#Merge FamilyRanks_ForGroup with the midpoint for goldilocks ranks into a new dataframe
GoldilocksFamilyRanks = FamilyRanks_ForGroup.merge(indicatorInformation[['Code','Ranking_GoldilocksMidpoint']], how = 'left', on = ['Code'], suffixes = ('_Goldi','_Info'))

#Calculate ranking and percentile based on distance from goldilocks mid point.
GoldilocksFamilyRanks['AbsoluteDifferenceFromGoldilocksMidPoint'] = GoldilocksFamilyRanks.apply(DifferenceFromGoldilocksMidPoint, axis = 1)
GoldilocksFamilyRanks = GoldilocksFamilyRanks[pd.notnull(GoldilocksFamilyRanks['AbsoluteDifferenceFromGoldilocksMidPoint'])]
GoldilocksFamilyRanks['FamilyRank_Goldi'] = GoldilocksFamilyRanks.groupby(['Code','Period','Family_Group'])['AbsoluteDifferenceFromGoldilocksMidPoint'].rank('dense',ascending = True).astype(int)
GoldilocksFamilyRanks['FamilyRank_Goldi_Pct'] = GoldilocksFamilyRanks.groupby(['Code','Period', 'Family_Group'])['AbsoluteDifferenceFromGoldilocksMidPoint'].rank('dense',ascending = True,pct = True).astype(float)


#Merge relevant columns from GoldilocksFamilyRanks into the main FamilyRanks dataframe.
FamilyRanks = FamilyRanks.merge(GoldilocksFamilyRanks[['Code','Period','Local_Authority','FamilyRank_Goldi','FamilyRank_Goldi_Pct']], how = 'left', on = ['Code','Period','Local_Authority'], suffixes = ('_FamilyRank','_Goldi'))

#Merge the ranking type from the indicatorInformation dataframe. This will allow us to select the correct ranking and percentile in the next steps.
FamilyRanks = FamilyRanks.merge(indicatorInformation[['Code','Ranking_Type']], how = 'left', on = ['Code'], suffixes = ('_FamilyAv','_Info'))


# Define functions needed to select correct ranking type and percentile type
def FamilyRank_select(df) : 
    if df['Ranking_Type'] == "Ascending" :
        return df['FamilyRank_Asc']
    elif df['Ranking_Type'] == "Descending" :
        return df['FamilyRank_Desc']
    elif df['Ranking_Type'] == "Goldilocks" :
        return df['FamilyRank_Goldi']
    else :
        return None

def FamilyRank_Pct_select(df) : 
    if df['Ranking_Type'] == "Ascending" :
        return df['FamilyRank_Asc_Pct']
    elif df['Ranking_Type'] == "Descending" :
        return df['FamilyRank_Desc_Pct']
    elif df['Ranking_Type'] == "Goldilocks" :
        return df['FamilyRank_Goldi_Pct']
    else :
        return None

# Apply functions above to create two new columns that contain the correct rank and percentile for each row
FamilyRanks['FamilyRank'] = FamilyRanks.apply(FamilyRank_select, axis = 1)
FamilyRanks['FamilyPct'] = FamilyRanks.apply(FamilyRank_Pct_select, axis = 1)

# Merge the relevant columns from FamilyRanks into the main all_LGBFData dataframe
FamilyRanks = FamilyRanks[['GSS Code','Local_Authority','Code','Period','FamilyRank','FamilyPct']]
all_LGBFData = all_LGBFData.merge(FamilyRanks,how = 'left', on = ['GSS Code','Local_Authority','Code','Period'], suffixes = ('_ScotAv','_Info'))

all_LGBFData

,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct
0,S12000033,Aberdeen City,CHN1,2010-11,5843.392536,6.953637e+07,11900.0,4960.336134,59028000.0,11900.0,22.0,0.68750,6.0,0.750
1,S12000034,Aberdeenshire,CHN1,2010-11,5708.665025,1.091097e+08,19113.0,4845.968712,92621000.0,19113.0,20.0,0.62500,5.0,0.625
2,S12000041,Angus,CHN1,2010-11,5508.700818,4.649894e+07,8441.0,4676.223196,39472000.0,8441.0,11.0,0.34375,3.0,0.375
3,S12000035,Argyll & Bute,CHN1,2010-11,6889.855421,4.001628e+07,5808.0,5848.657025,33969000.0,5808.0,29.0,0.90625,8.0,1.000
4,S12000005,Clackmannanshire,CHN1,2010-11,5209.750091,1.981789e+07,3804.0,4422.450053,16823000.0,3804.0,4.0,0.12500,3.0,0.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24536,S12000028,South Ayrshire,CLIM2,2018-19,5.700050,NaN,NaN,5.700050,NaN,NaN,19.0,0.59375,4.0,0.500
24537,S12000029,South Lanarkshire,CLIM2,2018-19,3.970788,NaN,NaN,3.970788,NaN,NaN,4.0,0.12500,2.0,0.250
24538,S12000030,Stirling,CLIM2,2018-19,6.437136,NaN,NaN,6.437136,NaN,NaN,25.0,0.78125,6.0,0.750
24539,S12000039,West Dunbartonshire,CLIM2,2018-19,4.201782,NaN,NaN,4.201782,NaN,NaN,7.0,0.21875,4.0,0.500


## 8 Adding Previous Period and Initial Row Data & Comparisons

### 8.1 Add previous and inital rows as dictionaries in their own columns

In [20]:
#Sort rows of the all_LGBFData dataframe to ensure that we are getting the correct previous and first rows in the subsequent steps. This may not be strictly necessary as the data should already be in the correct sort order. It is here as a safeguard in case any sorting needs to be done in previous steps at a later date.
all_LGBFData.sort_values(by = ['Local_Authority','Code','Period'],inplace = True)

#Define all variables that will be used to record the changes from previous/first.
Previouss = []
Previous = None
Firsts = []
First = None
First_Save = None
Local_Authority = ""
Code = ""
Period = ""
Real_Value = ""
Real_Numerator = ""
Real_Denominator = ""
Cash_Value = ""
Cash_Numerator = ""
Cash_Denominator = ""
ScotRank = ""
ScotPct = ""
FamilyRank = ""
FamilyPct = ""

# Loop over the all_LGBFDataframe and record previous and first into a python dictionary object. Save the objects for each row into the two list variables (Previouss and Firsts)
for row in all_LGBFData.itertuples() :
    # If the curently stored Local_Authority and Code are both equal to the current row then this is not the first row for this indicator and local authority combination. As such Previous is calculated using all of the currently stored values in the variables (these are written to at the end of each loop) and First is populated using the stored dictionary in First_Save
    if Local_Authority == row.Local_Authority and Code == row.Code :
        Previous = {
            'Real_Value' : Real_Value,
            'Real_Numerator' : Real_Numerator,
            'Real_Denominator' : Real_Denominator,
            'Cash_Value' : Cash_Value,
            'Cash_Numerator' : Cash_Numerator,
            'Cash_Denominator' : Cash_Denominator,
            'ScotRank' : ScotRank,
            'ScotPct' : ScotPct,
            'FamilyRank' : FamilyRank,
            'FamilyPct' : FamilyPct
            }
        First = First_Save

    # If the curently stored Local_Authority and Code are both not equal to the current row then this is the first row for this indicator and local authority combination. as such the Previous object is set to None and the First object is populated using this rows values.
    elif Local_Authority != row.Local_Authority or Code != row.Code :
        First_Save = {
            'Real_Value' : row.Real_Value,
            'Real_Numerator' : row.Real_Numerator,
            'Real_Denominator' : row.Real_Denominator,
            'Cash_Value' : row.Cash_Value,
            'Cash_Numerator' : row.Cash_Numerator,
            'Cash_Denominator' : row.Cash_Denominator,
            'ScotRank' : row.ScotRank,
            'ScotPct' : row.ScotPct,
            'FamilyRank' : row.FamilyRank,
            'FamilyPct' : row.FamilyPct
            }
        First = None
        Previous = None
    
    # Append the First and Previous into their respective list variables.
    Previouss.append(Previous)
    Firsts.append(First)

    # Set all other variables to their respective columns values in the current row. This is used to both evaluate the if criteria above and to populate the next previous object.
    Local_Authority = row.Local_Authority
    Code = row.Code
    Period = row.Period
    Real_Value = row.Real_Value
    Real_Numerator = row.Real_Numerator
    Real_Denominator = row.Real_Denominator
    Cash_Value = row.Cash_Value
    Cash_Numerator = row.Cash_Numerator
    Cash_Denominator = row.Cash_Denominator
    ScotRank = row.ScotRank
    ScotPct = row.ScotPct
    FamilyRank = row.FamilyRank
    FamilyPct = row.FamilyPct
    
# Assign the two list variables into appropriately titled columns within our all_LGBFData dataframe
all_LGBFData['Previous_Row'] = Previouss
all_LGBFData['First_Row'] = Firsts

all_LGBFData

,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct,Previous_Row,First_Row
12243,S12000033,Aberdeen City,C&L1,2010-11,0.388748,7.504010e+05,1922292.000,0.330000,637000.0,1922292.000,1.0,0.032258,1.0,0.125,None,None
12275,S12000033,Aberdeen City,C&L1,2011-12,0.881791,1.803308e+06,2045051.000,0.759883,1554000.0,2045051.000,1.0,0.031250,1.0,0.125,"{'Real_Value': 0.38874775512896714, 'Real_Nume...","{'Real_Value': 0.38874775512896714, 'Real_Nume..."
12307,S12000033,Aberdeen City,C&L1,2012-13,4.142972,8.964380e+06,2163756.000,3.643202,7883000.0,2163756.000,24.0,0.750000,5.0,0.625,"{'Real_Value': 0.8817910882543134, 'Real_Numer...","{'Real_Value': 0.38874775512896714, 'Real_Nume..."
12339,S12000033,Aberdeen City,C&L1,2013-14,3.882945,8.630187e+06,2222588.000,3.476128,7726000.0,2222588.000,23.0,0.718750,5.0,0.625,"{'Real_Value': 4.142971696712512, 'Real_Numera...","{'Real_Value': 0.38874775512896714, 'Real_Nume..."
12371,S12000033,Aberdeen City,C&L1,2014-15,3.330141,8.282520e+06,2487138.000,3.022349,7517000.0,2487138.000,20.0,0.625000,5.0,0.625,"{'Real_Value': 3.882944941630109, 'Real_Numera...","{'Real_Value': 0.38874775512896714, 'Real_Nume..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12114,S12000040,West Lothian,SW8,2015-16,485.210264,5.446000e+03,11.224,485.210264,5446.0,11.224,10.0,0.312500,3.0,0.375,"{'Real_Value': 623.8970588235294, 'Real_Numera...","{'Real_Value': 548.7757856178908, 'Real_Numera..."
12146,S12000040,West Lothian,SW8,2016-17,822.281625,9.536000e+03,11.597,822.281625,9536.0,11.597,20.0,0.625000,5.0,0.625,"{'Real_Value': 485.21026372059873, 'Real_Numer...","{'Real_Value': 548.7757856178908, 'Real_Numera..."
12178,S12000040,West Lothian,SW8,2017-18,1139.053501,1.364700e+04,11.981,1139.053501,13647.0,11.981,28.0,0.875000,8.0,1.000,"{'Real_Value': 822.2816245580755, 'Real_Numera...","{'Real_Value': 548.7757856178908, 'Real_Numera..."
12210,S12000040,West Lothian,SW8,2018-19,1214.250000,1.506400e+04,12.406,1214.250000,15064.0,12.406,27.0,0.843750,7.0,0.875,"{'Real_Value': 1139.0535013771805, 'Real_Numer...","{'Real_Value': 548.7757856178908, 'Real_Numera..."


### 8.2 Add comparisons to previous and first data

In [21]:
# Define a function that will return an aim adjusted percentage change between two indicator values. There are two niche cases here. One where previous and current values are both 0 resulting in 0% in all cases. Another where only the previous value is 0 resulting in None being returned as it is not possible to calculate % change from 0. Having looked at the dataset this has only occured 3 times and only affects Orkney and Eilean Siar for CHN20b. Further to this changes in percentage indicators are calculated using 100 as a denominator rather than previous. This is to avoid situations where very small percentages return 1000% or more change (which for our purposes seemed unreasonable to report).
def PercentChange_AimAdjusted (Previous,Current,Code) :
    Aim  = None
    SignedChange = None
    PercentChange = None
    GoldiMid = None
    IsPercentage = False

    if Previous == 0 and Current == 0 :
        PercentChange = 0
    
    indicatorInfo = indicatorInformation_dict[Code]
    Aim = indicatorInfo['Ranking_Type']
    GoldiMid = indicatorInfo['Ranking_GoldilocksMidpoint']
    IsPercentage = indicatorInfo['MeasureType'] == "Percentage"

    if IsPercentage == False and Previous != 0 :
        if Aim == "Ascending": 
            SignedChange = distance(Current,Previous)
            PercentChange = SignedChange/Previous
        if Aim == "Descending":
            SignedChange = -distance(Current,Previous)
            PercentChange = SignedChange/Previous
        if Aim == "Goldilocks":
            Current_DistGoldi = abs(distance(Current,GoldiMid))
            Previous_DistGoldi = abs(distance(Previous,GoldiMid))
            SignedChange = distance(Previous_DistGoldi,Current_DistGoldi)
            PercentChange = SignedChange/Previous_DistGoldi

    elif IsPercentage == True and Previous != 0 :
        if Aim == "Ascending": 
            SignedChange = distance(Current,Previous)
            PercentChange = SignedChange/100
        if Aim == "Descending":
            SignedChange = -distance(Current,Previous)
            PercentChange = SignedChange/100
        if Aim == "Goldilocks":
            Current_DistGoldi = abs(distance(Current,GoldiMid))
            Previous_DistGoldi = abs(distance(Previous,GoldiMid))
            SignedChange = distance(Previous_DistGoldi,Current_DistGoldi)
            PercentChange = SignedChange/100

    return PercentChange

# Define a function that calculates the changes in the current row by accessing values in the Previous_Row and First_Row python dictionaries.
def Changes(df) :
    
    # Set the intial value of the return variable to none. This allows us to test to see if there were any changes present for a row and then return None instead of a dictionary of None values if not.
    Changes = None

    # Define all variables that will contain all of the relevant changes for a row.
    ScotRank_ChangeSincePrevious = None
    ScotPct_ChangeSincePrevious = None
    FamilyRank_ChangeSincePrevious = None
    FamilyPct_ChangeSincePrevious = None
    Real_Value_ChangeSincePrevious = None
    Real_Numerator_ChangeSincePrevious = None
    Real_Denominator_ChangeSincePrevious = None
    Cash_Value_ChangeSincePrevious = None
    Cash_Numerator_ChangeSincePrevious = None
    Cash_Denominator_ChangeSincePrevious = None
    ScotRank_ChangeSinceFirst = None
    ScotPct_ChangeSinceFirst = None
    FamilyRank_ChangeSinceFirst = None
    FamilyPct_ChangeSinceFirst = None
    Real_Value_ChangeSinceFirst = None
    Real_Numerator_ChangeSinceFirst = None
    Real_Denominator_ChangeSinceFirst = None
    Cash_Value_ChangeSinceFirst = None
    Cash_Numerator_ChangeSinceFirst = None
    Cash_Denominator_ChangeSinceFirst = None
    PercentChange_AimAdjusted_SincePrevious = None
    PercentChange_AimAdjusted_SinceFirst = None

    # If the value currently in Previous_Row is not None then there exists a previous object to calculate the changes using.
    if df['Previous_Row'] != None :
        # Calculate all differences by comparing the current rows value to the same columns value in the Previous_Row dictionary. Ranks and Percentiles are always positive so the calculations are more simple. The other values use the distance function defined at the start of the notebook to determine the signed difference between the values (comparing a current value of -1 to a previous value of 2 will result in -3 difference.)
        ScotRank_ChangeSincePrevious = - (df['ScotRank'] - df['Previous_Row'].get('ScotRank'))
        ScotPct_ChangeSincePrevious = - (df['ScotPct'] - df['Previous_Row'].get('ScotPct'))
        FamilyRank_ChangeSincePrevious = - (df['FamilyRank'] - df['Previous_Row'].get('FamilyRank'))
        FamilyPct_ChangeSincePrevious = - (df['FamilyPct'] - df['Previous_Row'].get('FamilyPct'))
        Real_Value_ChangeSincePrevious = distance(df['Real_Value'],df['Previous_Row'].get('Real_Value'))
        Real_Numerator_ChangeSincePrevious = distance(df['Real_Numerator'],df['Previous_Row'].get('Real_Numerator'))
        Real_Denominator_ChangeSincePrevious = distance(df['Real_Denominator'],df['Previous_Row'].get('Real_Denominator'))
        Cash_Value_ChangeSincePrevious = distance(df['Cash_Value'],df['Previous_Row'].get('Cash_Value'))
        Cash_Numerator_ChangeSincePrevious = distance(df['Cash_Numerator'],df['Previous_Row'].get('Cash_Numerator'))
        Cash_Denominator_ChangeSincePrevious = distance(df['Cash_Denominator'],df['Previous_Row'].get('Cash_Denominator'))
        PercentChange_AimAdjusted_SincePrevious = PercentChange_AimAdjusted(df['Previous_Row'].get('Real_Value'),df['Real_Value'],df['Code'])
        #Set Changes to true to avoid creating a dictionary of None values
        Changes = True
    
    # If the value currently in First_Row is not None then there exists a previous object to calculate the changes using.
    if df['First_Row'] != None :
                # Calculate all differences by comparing the current rows value to the same columns value in the First_Row dictionary. Ranks and Percentiles are always positive so the calculations are more simple. The other values use the distance function defined at the start of the notebook to determine the signed difference between the values (comparing a current value of -1 to a previous value of 2 will result in -3 difference.)
        ScotRank_ChangeSinceFirst = - (df['ScotRank'] - df['First_Row'].get('ScotRank'))
        ScotPct_ChangeSinceFirst = - (df['ScotPct'] - df['First_Row'].get('ScotPct'))
        FamilyRank_ChangeSinceFirst = - (df['FamilyRank'] - df['First_Row'].get('FamilyRank'))
        FamilyPct_ChangeSinceFirst = - (df['FamilyPct'] - df['First_Row'].get('FamilyPct'))
        Real_Value_ChangeSinceFirst = distance(df['Real_Value'],df['First_Row'].get('Real_Value'))
        Real_Numerator_ChangeSinceFirst = distance(df['Real_Numerator'],df['First_Row'].get('Real_Numerator'))
        Real_Denominator_ChangeSinceFirst = distance(df['Real_Denominator'],df['First_Row'].get('Real_Denominator'))
        Cash_Value_ChangeSinceFirst = distance(df['Cash_Value'],df['First_Row'].get('Cash_Value'))
        Cash_Numerator_ChangeSinceFirst = distance(df['Cash_Numerator'],df['First_Row'].get('Cash_Numerator'))
        Cash_Denominator_ChangeSinceFirst = distance(df['Cash_Denominator'],df['First_Row'].get('Cash_Denominator'))
        PercentChange_AimAdjusted_SinceFirst = PercentChange_AimAdjusted(df['First_Row'].get('Real_Value'),df['Real_Value'],df['Code'])
        #Set Changes to true to avoid creating a dictionary of None values
        Changes = True

    # If there were changes recorded in the previous steps then write these changes into a python dictionary and assign this to Changes
    if Changes != None :
        Changes = {
            "ScotRank_ChangeSincePrevious" : ScotRank_ChangeSincePrevious,
            "ScotPct_ChangeSincePrevious" : ScotPct_ChangeSincePrevious,
            "FamilyRank_ChangeSincePrevious" : FamilyRank_ChangeSincePrevious,
            "FamilyPct_ChangeSincePrevious" : FamilyPct_ChangeSincePrevious,
            "ScotRank_ChangeSinceFirst" : ScotRank_ChangeSinceFirst,
            "ScotPct_ChangeSinceFirst" : ScotPct_ChangeSinceFirst,
            "FamilyRank_ChangeSinceFirst" : FamilyRank_ChangeSinceFirst,
            "FamilyPct_ChangeSinceFirst" : FamilyPct_ChangeSinceFirst,
            "Real_Value_ChangeSincePrevious" : Real_Value_ChangeSincePrevious,
            "Real_Numerator_ChangeSincePrevious" : Real_Numerator_ChangeSincePrevious,
            "Real_Denominator_ChangeSincePrevious" : Real_Denominator_ChangeSincePrevious,
            "Cash_Value_ChangeSincePrevious" : Cash_Value_ChangeSincePrevious,
            "Cash_Numerator_ChangeSincePrevious" : Cash_Numerator_ChangeSincePrevious,
            "Cash_Denominator_ChangeSincePrevious" : Cash_Denominator_ChangeSincePrevious,
            "Real_Value_ChangeSinceFirst" : Real_Value_ChangeSinceFirst,
            "Real_Numerator_ChangeSinceFirst" : Real_Numerator_ChangeSinceFirst,
            "Real_Denominator_ChangeSinceFirst" : Real_Denominator_ChangeSinceFirst,
            "Cash_Value_ChangeSinceFirst" : Cash_Value_ChangeSinceFirst,
            "Cash_Numerator_ChangeSinceFirst" : Cash_Numerator_ChangeSinceFirst,
            "Cash_Denominator_ChangeSinceFirst" : Cash_Denominator_ChangeSinceFirst,
            "PercentChange_AimAdjusted_SincePrevious" : PercentChange_AimAdjusted_SincePrevious,
            "PercentChange_AimAdjusted_SinceFirst" : PercentChange_AimAdjusted_SinceFirst
        }

    return Changes

#Apply the above function and output it to a new column named Changes.
all_LGBFData['Changes'] = all_LGBFData.apply(Changes, axis = 1)
all_LGBFData

,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct,Previous_Row,First_Row,Changes
12243,S12000033,Aberdeen City,C&L1,2010-11,0.388748,7.504010e+05,1922292.000,0.330000,637000.0,1922292.000,1.0,0.032258,1.0,0.125,None,None,None
12275,S12000033,Aberdeen City,C&L1,2011-12,0.881791,1.803308e+06,2045051.000,0.759883,1554000.0,2045051.000,1.0,0.031250,1.0,0.125,"{'Real_Value': 0.38874775512896714, 'Real_Nume...","{'Real_Value': 0.38874775512896714, 'Real_Nume...","{'ScotRank_ChangeSincePrevious': -0.0, 'ScotPc..."
12307,S12000033,Aberdeen City,C&L1,2012-13,4.142972,8.964380e+06,2163756.000,3.643202,7883000.0,2163756.000,24.0,0.750000,5.0,0.625,"{'Real_Value': 0.8817910882543134, 'Real_Numer...","{'Real_Value': 0.38874775512896714, 'Real_Nume...","{'ScotRank_ChangeSincePrevious': -23.0, 'ScotP..."
12339,S12000033,Aberdeen City,C&L1,2013-14,3.882945,8.630187e+06,2222588.000,3.476128,7726000.0,2222588.000,23.0,0.718750,5.0,0.625,"{'Real_Value': 4.142971696712512, 'Real_Numera...","{'Real_Value': 0.38874775512896714, 'Real_Nume...","{'ScotRank_ChangeSincePrevious': 1.0, 'ScotPct..."
12371,S12000033,Aberdeen City,C&L1,2014-15,3.330141,8.282520e+06,2487138.000,3.022349,7517000.0,2487138.000,20.0,0.625000,5.0,0.625,"{'Real_Value': 3.882944941630109, 'Real_Numera...","{'Real_Value': 0.38874775512896714, 'Real_Nume...","{'ScotRank_ChangeSincePrevious': 3.0, 'ScotPct..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12114,S12000040,West Lothian,SW8,2015-16,485.210264,5.446000e+03,11.224,485.210264,5446.0,11.224,10.0,0.312500,3.0,0.375,"{'Real_Value': 623.8970588235294, 'Real_Numera...","{'Real_Value': 548.7757856178908, 'Real_Numera...","{'ScotRank_ChangeSincePrevious': -1.0, 'ScotPc..."
12146,S12000040,West Lothian,SW8,2016-17,822.281625,9.536000e+03,11.597,822.281625,9536.0,11.597,20.0,0.625000,5.0,0.625,"{'Real_Value': 485.21026372059873, 'Real_Numer...","{'Real_Value': 548.7757856178908, 'Real_Numera...","{'ScotRank_ChangeSincePrevious': -10.0, 'ScotP..."
12178,S12000040,West Lothian,SW8,2017-18,1139.053501,1.364700e+04,11.981,1139.053501,13647.0,11.981,28.0,0.875000,8.0,1.000,"{'Real_Value': 822.2816245580755, 'Real_Numera...","{'Real_Value': 548.7757856178908, 'Real_Numera...","{'ScotRank_ChangeSincePrevious': -8.0, 'ScotPc..."
12210,S12000040,West Lothian,SW8,2018-19,1214.250000,1.506400e+04,12.406,1214.250000,15064.0,12.406,27.0,0.843750,7.0,0.875,"{'Real_Value': 1139.0535013771805, 'Real_Numer...","{'Real_Value': 548.7757856178908, 'Real_Numera...","{'ScotRank_ChangeSincePrevious': 1.0, 'ScotPct..."


### 8.3 Convert Python Dictionaries to JSON
This is to make the final file more usable in BI products such as PowerBI

In [22]:
def PreviousConvertToJson (df) :
    Previous_Row = simplejson.dumps(df['Previous_Row'],ignore_nan=True)    
    return Previous_Row

def FirstConvertToJson (df) :
    First_Row = simplejson.dumps(df['First_Row'],ignore_nan=True)    
    return First_Row

def ChangesConvertToJson (df) :
    Changes = simplejson.dumps(df['Changes'],ignore_nan=True)    
    return Changes

all_LGBFData['Previous_Row'] = all_LGBFData.apply(PreviousConvertToJson, axis = 1)
all_LGBFData['First_Row'] = all_LGBFData.apply(FirstConvertToJson, axis = 1)
all_LGBFData['Changes'] = all_LGBFData.apply(ChangesConvertToJson, axis = 1)

# Merge sortable codes into dataframe
all_LGBFData = all_LGBFData.merge(indicatorInformation[['Code','Code_Sortable']], how = 'left', on = ['Code'], suffixes = ('ISScot','_info'))

# Add relationship Key Columns
all_LGBFData['Key_CodePeriod'] = all_LGBFData['Code_Sortable'] + all_LGBFData['Period']
all_LGBFData['Key_CodePeriodLA'] = all_LGBFData['Code_Sortable'] + all_LGBFData['Period'] + all_LGBFData['Local_Authority']
all_LGBFData = all_LGBFData.merge(indicatorInformation[['Code_Sortable','FamilyGrouping']], how = 'left', on = ['Code_Sortable'], suffixes = ('_all','_info'))
all_LGBFData = all_LGBFData.rename(columns = {'FamilyGrouping' : 'Type'})
all_LGBFData = all_LGBFData.merge(Family_Groups,how ='left', on = ['Local_Authority','Type'], suffixes = ('_all','_group'))
all_LGBFData['Key_CodePeriodFamilyGroup'] = all_LGBFData['Code_Sortable'] + all_LGBFData['Period'] + all_LGBFData['Family_Group']

# Reorder and rename columns appropriately
all_LGBFData = all_LGBFData[['Key_CodePeriod','Key_CodePeriodFamilyGroup','Key_CodePeriodLA','GSS Code','Local_Authority','Code_Sortable','Period','Real_Value','Real_Numerator','Real_Denominator','Cash_Value','Cash_Numerator','Cash_Denominator','ScotRank','ScotPct','FamilyRank','FamilyPct','Previous_Row','First_Row','Changes']]
all_LGBFData = all_LGBFData.rename(columns = {'Code_Sortable' : 'Code'})

all_LGBFData

,Key_CodePeriod,Key_CodePeriodFamilyGroup,Key_CodePeriodLA,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct,Previous_Row,First_Row,Changes
0,C&L 012010-11,C&L 012010-11Family Group 4,C&L 012010-11Aberdeen City,S12000033,Aberdeen City,C&L 01,2010-11,0.388748,7.504010e+05,1922292.000,0.330000,637000.0,1922292.000,1.0,0.032258,1.0,0.125,null,null,null
1,C&L 012011-12,C&L 012011-12Family Group 4,C&L 012011-12Aberdeen City,S12000033,Aberdeen City,C&L 01,2011-12,0.881791,1.803308e+06,2045051.000,0.759883,1554000.0,2045051.000,1.0,0.031250,1.0,0.125,"{""Real_Value"": 0.38874775512896714, ""Real_Nume...","{""Real_Value"": 0.38874775512896714, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": -0.0, ""ScotPc..."
2,C&L 012012-13,C&L 012012-13Family Group 4,C&L 012012-13Aberdeen City,S12000033,Aberdeen City,C&L 01,2012-13,4.142972,8.964380e+06,2163756.000,3.643202,7883000.0,2163756.000,24.0,0.750000,5.0,0.625,"{""Real_Value"": 0.8817910882543134, ""Real_Numer...","{""Real_Value"": 0.38874775512896714, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": -23.0, ""ScotP..."
3,C&L 012013-14,C&L 012013-14Family Group 4,C&L 012013-14Aberdeen City,S12000033,Aberdeen City,C&L 01,2013-14,3.882945,8.630187e+06,2222588.000,3.476128,7726000.0,2222588.000,23.0,0.718750,5.0,0.625,"{""Real_Value"": 4.142971696712512, ""Real_Numera...","{""Real_Value"": 0.38874775512896714, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": 1.0, ""ScotPct..."
4,C&L 012014-15,C&L 012014-15Family Group 4,C&L 012014-15Aberdeen City,S12000033,Aberdeen City,C&L 01,2014-15,3.330141,8.282520e+06,2487138.000,3.022349,7517000.0,2487138.000,20.0,0.625000,5.0,0.625,"{""Real_Value"": 3.882944941630109, ""Real_Numera...","{""Real_Value"": 0.38874775512896714, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": 3.0, ""ScotPct..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24536,SW 082015-16,SW 082015-16Family Group 3,SW 082015-16West Lothian,S12000040,West Lothian,SW 08,2015-16,485.210264,5.446000e+03,11.224,485.210264,5446.0,11.224,10.0,0.312500,3.0,0.375,"{""Real_Value"": 623.8970588235294, ""Real_Numera...","{""Real_Value"": 548.7757856178908, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -1.0, ""ScotPc..."
24537,SW 082016-17,SW 082016-17Family Group 3,SW 082016-17West Lothian,S12000040,West Lothian,SW 08,2016-17,822.281625,9.536000e+03,11.597,822.281625,9536.0,11.597,20.0,0.625000,5.0,0.625,"{""Real_Value"": 485.21026372059873, ""Real_Numer...","{""Real_Value"": 548.7757856178908, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -10.0, ""ScotP..."
24538,SW 082017-18,SW 082017-18Family Group 3,SW 082017-18West Lothian,S12000040,West Lothian,SW 08,2017-18,1139.053501,1.364700e+04,11.981,1139.053501,13647.0,11.981,28.0,0.875000,8.0,1.000,"{""Real_Value"": 822.2816245580755, ""Real_Numera...","{""Real_Value"": 548.7757856178908, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -8.0, ""ScotPc..."
24539,SW 082018-19,SW 082018-19Family Group 3,SW 082018-19West Lothian,S12000040,West Lothian,SW 08,2018-19,1214.250000,1.506400e+04,12.406,1214.250000,15064.0,12.406,27.0,0.843750,7.0,0.875,"{""Real_Value"": 1139.0535013771805, ""Real_Numer...","{""Real_Value"": 548.7757856178908, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": 1.0, ""ScotPct..."


## 9 Split Last Values Into Separate Dataframe

In [23]:
LastValues = all_LGBFData.copy(deep = True)
LastValues.sort_values(by = ['Code','Period','Local_Authority'],inplace = True)
LastValues = LastValues.groupby(['Code','Local_Authority']).tail(1)
LastValues['Key_CodePeriodLA'] = LastValues['Code'] + LastValues['Period'] + LastValues['Local_Authority']
LastValues = LastValues[['Key_CodePeriodLA','GSS Code','Local_Authority','Code','Period','Real_Value','Real_Numerator','Real_Denominator','Cash_Value','Cash_Numerator','Cash_Denominator','ScotRank','ScotPct','FamilyRank','FamilyPct','Previous_Row','First_Row','Changes']]
LastValues

,Key_CodePeriodLA,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct,Previous_Row,First_Row,Changes
9,C&L 012019-20Aberdeen City,S12000033,Aberdeen City,C&L 01,2019-20,3.449565,3950000.0,1145072.000,3.449565,3950000.0,1145072.000,24.0,0.75000,6.0,0.750,"{""Real_Value"": 3.6464745264559357, ""Real_Numer...","{""Real_Value"": 0.38874775512896714, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": 3.0, ""ScotPct..."
791,C&L 012019-20Aberdeenshire,S12000034,Aberdeenshire,C&L 01,2019-20,2.980393,6615000.0,2219506.000,2.980393,6615000.0,2219506.000,21.0,0.65625,6.0,0.750,"{""Real_Value"": 2.567573317890463, ""Real_Numera...","{""Real_Value"": 5.277545281750827, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -2.0, ""ScotPc..."
1570,C&L 012019-20Angus,S12000041,Angus,C&L 01,2019-20,2.870049,3509000.0,1222627.000,2.870049,3509000.0,1222627.000,20.0,0.62500,6.0,0.750,"{""Real_Value"": 2.546556191738002, ""Real_Numera...","{""Real_Value"": 2.3089260607659865, ""Real_Numer...","{""ScotRank_ChangeSincePrevious"": -3.0, ""ScotPc..."
2353,C&L 012019-20Argyll & Bute,S12000035,Argyll & Bute,C&L 01,2019-20,2.791059,1589000.0,569318.000,2.791059,1589000.0,569318.000,19.0,0.59375,5.0,0.625,"{""Real_Value"": 2.9976357511271754, ""Real_Numer...","{""Real_Value"": 2.8508168709457586, ""Real_Numer...","{""ScotRank_ChangeSincePrevious"": 1.0, ""ScotPct..."
3089,C&L 012019-20Clackmannanshire,S12000005,Clackmannanshire,C&L 01,2019-20,0.836885,521000.0,622547.000,0.836885,521000.0,622547.000,4.0,0.12500,1.0,0.125,"{""Real_Value"": 0.7650417846260176, ""Real_Numer...","{""Real_Value"": 3.5340705011724283, ""Real_Numer...","{""ScotRank_ChangeSincePrevious"": -1.0, ""ScotPc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21409,SW 082019-20South Ayrshire,S12000028,South Ayrshire,SW 08,2019-20,1699.446088,22397.0,13.179,1699.446088,22397.0,13.179,32.0,1.00000,8.0,1.000,"{""Real_Value"": 1353.51, ""Real_Numerator"": 1726...","{""Real_Value"": 629.4619982920581, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -2.0, ""ScotPc..."
22192,SW 082019-20South Lanarkshire,S12000029,South Lanarkshire,SW 08,2019-20,1155.685152,31905.0,27.607,1155.685152,31905.0,27.607,27.0,0.84375,7.0,0.875,"{""Real_Value"": 1024.66, ""Real_Numerator"": 2755...","{""Real_Value"": 1368.7569810116481, ""Real_Numer...","{""ScotRank_ChangeSincePrevious"": -4.0, ""ScotPc..."
22974,SW 082019-20Stirling,S12000030,Stirling,SW 08,2019-20,848.781653,7106.0,8.372,848.781653,7106.0,8.372,22.0,0.68750,6.0,0.750,"{""Real_Value"": 540.34, ""Real_Numerator"": 4400....","{""Real_Value"": 837.0218579234972, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -11.0, ""ScotP..."
23757,SW 082019-20West Dunbartonshire,S12000039,West Dunbartonshire,SW 08,2019-20,541.008803,3872.0,7.157,541.008803,3872.0,7.157,15.0,0.46875,4.0,0.500,"{""Real_Value"": 331.96, ""Real_Numerator"": 2342....","{""Real_Value"": 705.2646930144379, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -9.0, ""ScotPc..."


## 10 Format Scottish Values

In [24]:
 # Convert value columns to numeric only using errors = 'coerce' to force any text values (dna, *, N/A etc.) to return NaN.
ISScottishValues[['Real_Value','Cash_Value']] = ISScottishValues[['Real_Value','Cash_Value']].apply(pd.to_numeric,errors='coerce')

# Remove rows which contain null or NaN in the Real_Value column as they are non-datapoints and could interfere with average, ranking and quartile calculations later.
ISScottishValues = ISScottishValues[pd.notnull(ISScottishValues['Real_Value'])]

# Convert Percentages to proper decimal percentages
ISScottishValues = ISScottishValues.merge(indicatorInformation[['Code','MeasureType']], how = 'left', on = ['Code'], suffixes = ('_all','_info'))

# These steps use the functions defined in step 5.5
ISScottishValues['Real_Value'] = ISScottishValues.apply(ConvertPercent_Real, axis = 1)
ISScottishValues['Cash_Value'] = ISScottishValues.apply(ConvertPercent_Cash, axis = 1)

ISScottishValues = ISScottishValues.drop(columns = ['MeasureType'])

# Merge sortable codes into dataframe
ISScottishValues = ISScottishValues.merge(indicatorInformation[['Code','Code_Sortable']], how = 'left', on = ['Code'], suffixes = ('ISScot','_info'))

# Add relationship Key Columns
ISScottishValues['Key_CodePeriod'] = ISScottishValues['Code_Sortable'] + ISScottishValues['Period']

# Reorder and rename columns appropriately
ISScottishValues = ISScottishValues[['Key_CodePeriod','Code_Sortable','Period','Real_Value','Cash_Value']]
ISScottishValues = ISScottishValues.rename(columns = {'Real_Value' : 'IS_Scot_Real_Value', 'Cash_Value' : 'IS_Scot_Cash_Value','Code_Sortable' : 'Code'})

ISScottishValues

,Key_CodePeriod,Code,Period,IS_Scot_Real_Value,IS_Scot_Cash_Value
0,CHN 012010-11,CHN 01,2010-11,5695.149739,4834.495863
1,CHN 012011-12,CHN 01,2011-12,5536.860202,4771.387895
2,CHN 012012-13,CHN 01,2012-13,5383.821439,4734.367021
3,CHN 012013-14,CHN 01,2013-14,5268.210602,4716.258860
4,CHN 012014-15,CHN 01,2014-15,5116.458029,4643.564279
...,...,...,...,...,...
778,CLIM 022014-15,CLIM 02,2014-15,5.729027,5.729027
779,CLIM 022015-16,CLIM 02,2015-16,5.462573,5.462573
780,CLIM 022016-17,CLIM 02,2016-17,5.185145,5.185145
781,CLIM 022017-18,CLIM 02,2017-18,5.011539,5.011539


## 11 Basic Verification Checks

In [25]:
# Default position is to assume checks are passed. If any of the checks (excluding numerator denominator checks) are failed below this will be changed and the final csv's will not be output. The numerator denominator errors should be checked at each refresh. The known errors (which exist within the LGBF raw data file) will be identified in the readme in the Error Outputs folder.
ChecksFailed = False

# ScotRank should be between 32 and 1 and should not contain any NaN values
if  not all_LGBFData['ScotRank'].between(1,8).any() or all_LGBFData['ScotRank'].isnull().values.any() :

    ChecksFailed = True
    maxrank = str(max(all_LGBFData['ScotRank']))
    minrank = str(min(all_LGBFData['ScotRank']))
    countnull = str(all_LGBFData['ScotRank'].isna().sum())

    print("ScotRank checks failed : Max - " + maxrank + ", Min - " + minrank + ", Count of Null - " + countnull)

# FamilyRank should be between 8 and 1 and should not contain any NaN values
if  not all_LGBFData['FamilyRank'].between(1,8).any() or all_LGBFData['FamilyRank'].isnull().values.any() :

    ChecksFailed = True
    maxrank = str(max(all_LGBFData['FamilyRank']))
    minrank = str(min(all_LGBFData['FamilyRank']))
    countnull = str(all_LGBFData['FamilyRank'].isna().sum())

    print("FamilyRank checks failed : Max - " + maxrank + ", Min - " + minrank + ", Count of Null - " + countnull)

# Code, Local_Authority, Period, Real_Value and Cash_Value should not contain any null values
if  all_LGBFData[['Code','Local_Authority','Period','Real_Value','Cash_Value',]].isnull().values.any() : 

    ChecksFailed = True
    countnullCode = str(all_LGBFData['Code'].isna().sum())
    countnullLocal_Authority = str(all_LGBFData['Local_Authority'].isna().sum())
    countnullPeriod = str(all_LGBFData['Period'].isna().sum())
    countnullReal_Value = str(all_LGBFData['Real_Value'].isna().sum())
    countnullCash_Value = str(all_LGBFData['Cash_Value'].isna().sum())

    print("Null values found : Code - " + countnullCode + ", Local_Authority - " + countnullLocal_Authority + ", Period - " + countnullPeriod + ", Real_Value - " + countnullReal_Value + ", Cash_Value - " + countnullCash_Value)

# Value should equal numerator/denominator for both cash and real - These errors have been checked and exist in the original raw data file.
NumDenCheck = all_LGBFData.copy(deep = True)
NumDenCheck = NumDenCheck[pd.notnull(NumDenCheck['Real_Numerator'])]

Real_NumDenDivide_Checks = []
Real_NumDenDivide_Check = None
Cash_NumDenDivide_Checks = []
Cash_NumDenDivide_Check = None
FailReferences = []

for row in NumDenCheck.itertuples() : 
    if row.Real_Value == 0 or math.isnan(row.Real_Denominator) or math.isnan(row.Real_Numerator) :
        Real_NumDenDivide_Check = None
    else :
        if math.isclose(row.Real_Numerator/row.Real_Denominator, row.Real_Value, rel_tol = 0.02): 
            Real_NumDenDivide_Check = True
        else : 
            Real_NumDenDivide_Check = False
            FailReferences.append("Real;" + row.Code + ";" + row.Period + ";" + row.Local_Authority + ";" + str(row.Real_Value) + ";" + str(row.Real_Numerator) + ";" + str(row.Real_Denominator))

        if math.isclose(row.Cash_Numerator/row.Cash_Denominator, row.Cash_Value, rel_tol = 0.02): 
            Cash_NumDenDivide_Check = True
        else : 
            Cash_NumDenDivide_Check = False
            FailReferences.append("Cash;" + row.Code + ";" + row.Period + ";" + row.Local_Authority + ";" + str(row.Cash_Value) + ";" + str(row.Cash_Numerator) + ";" + str(row.Cash_Denominator))

    Real_NumDenDivide_Checks.append(Real_NumDenDivide_Check)
    Cash_NumDenDivide_Checks.append(Cash_NumDenDivide_Check)

if False in Real_NumDenDivide_Checks or False in Cash_NumDenDivide_Checks : 
    print("Numerator/Denominator values check failed : See Error Outputs for csv of failures")
    FailReferences = sorted(list(set(FailReferences)))
    FailReferences = pd.DataFrame([sub.split(";") for sub in FailReferences])
    FailReferences = FailReferences.rename(columns = {0 : 'Type', 1 : 'Code', 2 : 'Period', 3 : 'Local Authority', 4 : 'Value', 5 : 'Numerator', 6 : 'Denominator'})
    FailReferences.to_csv("Error Outputs//Numerator Denominator Fail References.csv", index = False, encoding='utf-8-sig')

Numerator/Denominator values check failed : See Error Outputs for csv of failures


## 12 Output All Final Tables

In [26]:
if ChecksFailed == False :
    all_LGBFData.to_csv("Data Files//Indicator Data.csv", index = False, encoding='utf-8-sig')
    LastValues.to_csv("Data Files//Latest Values.csv", index = False, encoding='utf-8-sig')
    FamilyAverages.to_csv("Data Files//Family Averages.csv", index = False, encoding='utf-8-sig')
    ScottishAverages.to_csv("Data Files//Scottish Averages.csv", index = False, encoding='utf-8-sig')
    ISScottishValues.to_csv("Data Files//Scottish Values.csv", index = False, encoding='utf-8-sig')
    indicatorInformation.to_csv("Data Files//Indicator Information.csv", index = False, encoding='utf-8-sig')
    Family_Groups.to_csv("Data Files//Family Groups.csv", index = False, encoding='utf-8-sig')
else :
    print("Checks failed! Check output from section 11 for detail")